In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient as id_patient,
    d.case_id,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''

sdata = read_sql_query(query,engine,parse_dates=True)
sdata.id_patient = sdata.id_patient.astype(Int32Dtype())

# close the pool of connection
engine.dispose()

True

In [10]:
data = read_excel("./school_payed_2022_06_28.xlsx")
#datim=read_excel("./datim_verification_28_06_2022.xlsx",sheet_name="table_global")

In [11]:
sdata.case_id.count()
data.code.count()
datim.id_patient.count()

70435

10

15487

In [8]:
sp = sdata[sdata.code.isin(data.code)]
sp.code.count()

10

In [18]:
sp.age =  sp.age.astype(Int32Dtype())

def age_range(age):
    if age >=10 and age <=14:
        return "10-14"
    elif age >=15 and age <=19:
        return "15-19"
    elif age >=20 and age <=24:
        return "20-24"
    else:
        return "no"
    

sp['age_range'] = sp.age.map(age_range)

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_31314/2074706348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp.age =  sp.age.astype(Int32Dtype())
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_31314/2074706348.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp['age_range'] = sp.age.map(age_range)


In [21]:
sp.pivot_table(
    values="id_patient",
    columns="age_range",
    aggfunc="count",
    fill_value=0
)

age_range,10-14,15-19,20-24
id_patient,6,3,1
